In [ ]:
from dotenv import load_dotenv
load_dotenv('values.env')
from langsmith import traceable

In [ ]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from langchain.tools import tool
import os.path
import pickle
from typing import Optional

SCOPES = ['https://www.googleapis.com/auth/calendar']

@traceable
def get_calendar_service():
    """Authenticate and return calendar service."""
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_961240229011-6vncaf9dgjrre9k9no49l18h7v5k91a5.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return build('calendar', 'v3', credentials=creds)

@tool
@traceable
def list_calendars() -> str:
    """
    List all available Google calendars.
    
    Returns:
    
        List of calendar names and IDs
    """
    try:
        service = get_calendar_service()
        calendars = service.calendarList().list().execute()
        
        result = "Available calendars:\n"
        for cal in calendars.get('items', []):
            result += f"- {cal['summary']} (ID: {cal['id']})\n"
        return result
    except Exception as e:
        return f"Error listing calendars: {str(e)}"

@tool
@traceable
def create_calendar_event(
    summary: str,
    start_time: str,
    end_time: str,
    calendar_id: str = "primary",
    description: Optional[str] = None,
    location: Optional[str] = None,
    timezone: str = "America/New_York"
) -> str:
    """
    Create a Google Calendar event.
    
    Args:
        summary: Event title
        start_time: Start time in ISO 8601 format (e.g., '2025-10-15T10:00:00')
        end_time: End time in ISO 8601 format (e.g., '2025-10-15T11:00:00')
        calendar_id: Calendar ID or 'primary' (default: 'primary')
        description: Event description (optional)
        location: Event location (optional)
        timezone: Timezone (default: America/New_York)
    
    Returns:
        Success message with event link
    """
    try:
        service = get_calendar_service()
        
        event = {
            'summary': summary,
            'start': {
                'dateTime': start_time,
                'timeZone': timezone,
            },
            'end': {
                'dateTime': end_time,
                'timeZone': timezone,
            },
        }
        
        if description:
            event['description'] = description
        if location:
            event['location'] = location
        
        created_event = service.events().insert(calendarId=calendar_id, body=event).execute()
        return f"Event created successfully: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating event: {str(e)}"

# Example usage
if __name__ == '__main__':
    result = create_calendar_event.invoke({
        "summary": "Team Meeting",
        "start_time": "2025-10-15T10:00:00",
        "end_time": "2025-10-15T11:00:00",
        "description": "Monthly team sync",
        "location": "Conference Room A"
    })
    print(result)